In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator,img_to_array

from PIL import Image
import numpy as np
import os
import pathlib
import cv2

In [3]:
img_gen = ImageDataGenerator(dtype='float32')

In [4]:
train_image_gen = img_gen.flow_from_directory("eyes",target_size=(32,32),color_mode='grayscale',batch_size=1)

Found 2423 images belonging to 2 classes.


In [17]:

for i in range(0,2423):
    print(train_image_gen[i][1])


    
#print(train_image_gen[2422][0][0][0][0].shape)

AttributeError: 'DirectoryIterator' object has no attribute 'shape'

In [20]:
#success
images_array = train_image_gen[0][0]
labels_array = train_image_gen[0][1]
for i in range(1,2423):
    images_array = np.append(images_array,train_image_gen[i][0])
    labels_array = np.append(labels_array,train_image_gen[i][1])


images_array = images_array.reshape(2423,32,32,1)
labels_array = labels_array.reshape(2423,2)

In [21]:
print(images_array.shape)
print(labels_array.shape)


(2423, 32, 32, 1)
(2423, 2)


In [22]:
print(images_array)
print(labels_array)

[[[[ 53.]
   [ 52.]
   [ 56.]
   ...
   [  9.]
   [ 20.]
   [ 45.]]

  [[ 50.]
   [ 51.]
   [ 56.]
   ...
   [ 10.]
   [ 16.]
   [ 42.]]

  [[ 52.]
   [ 53.]
   [ 60.]
   ...
   [ 17.]
   [ 13.]
   [ 35.]]

  ...

  [[ 66.]
   [ 62.]
   [ 68.]
   ...
   [ 11.]
   [ 15.]
   [ 31.]]

  [[ 64.]
   [ 60.]
   [ 69.]
   ...
   [ 15.]
   [ 22.]
   [ 40.]]

  [[ 60.]
   [ 62.]
   [ 69.]
   ...
   [ 20.]
   [ 34.]
   [ 47.]]]


 [[[191.]
   [145.]
   [ 47.]
   ...
   [ 38.]
   [155.]
   [ 69.]]

  [[192.]
   [ 55.]
   [ 41.]
   ...
   [ 56.]
   [ 56.]
   [122.]]

  [[142.]
   [ 42.]
   [ 42.]
   ...
   [ 39.]
   [ 35.]
   [127.]]

  ...

  [[189.]
   [192.]
   [ 93.]
   ...
   [140.]
   [ 69.]
   [185.]]

  [[195.]
   [193.]
   [136.]
   ...
   [109.]
   [ 79.]
   [ 73.]]

  [[187.]
   [186.]
   [175.]
   ...
   [ 70.]
   [ 75.]
   [ 73.]]]


 [[[169.]
   [162.]
   [174.]
   ...
   [170.]
   [189.]
   [205.]]

  [[165.]
   [165.]
   [187.]
   ...
   [169.]
   [186.]
   [210.]]

  [[165.]
   [17

In [128]:
images_list = [[[[]]]]

a=[[[]]]
b=[[]]
c=[]
for i in range(0,2423):
    for j in range(0,32):
        for k in range(0,32):
            c=[]
            c.append(train_image_gen[i][j][k][0][0])
            b=[[]]
            b.append(c)
        a=[[[]]]
        a.append(b)
    images_list = images_list.append(a)

    
images_array=np.array(images_list)

print(images_array.shape)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [72]:
print(images_list[2422])

[[[[109.]
   [101.]
   [ 59.]
   ...
   [ 35.]
   [ 37.]
   [ 28.]]

  [[110.]
   [105.]
   [ 46.]
   ...
   [ 48.]
   [ 38.]
   [ 43.]]

  [[109.]
   [ 89.]
   [ 36.]
   ...
   [ 50.]
   [ 43.]
   [ 35.]]

  ...

  [[ 40.]
   [ 41.]
   [ 42.]
   ...
   [163.]
   [149.]
   [132.]]

  [[ 41.]
   [ 42.]
   [ 43.]
   ...
   [146.]
   [135.]
   [118.]]

  [[ 39.]
   [ 41.]
   [ 43.]
   ...
   [129.]
   [116.]
   [105.]]]]


In [73]:
print(images_array[2422])

[[[[109.]
   [101.]
   [ 59.]
   ...
   [ 35.]
   [ 37.]
   [ 28.]]

  [[110.]
   [105.]
   [ 46.]
   ...
   [ 48.]
   [ 38.]
   [ 43.]]

  [[109.]
   [ 89.]
   [ 36.]
   ...
   [ 50.]
   [ 43.]
   [ 35.]]

  ...

  [[ 40.]
   [ 41.]
   [ 42.]
   ...
   [163.]
   [149.]
   [132.]]

  [[ 41.]
   [ 42.]
   [ 43.]
   ...
   [146.]
   [135.]
   [118.]]

  [[ 39.]
   [ 41.]
   [ 43.]
   ...
   [129.]
   [116.]
   [105.]]]]


In [74]:
images_array = images_array/128

In [75]:
#print(images_array[2422])

[[[[0.8515625]
   [0.7890625]
   [0.4609375]
   ...
   [0.2734375]
   [0.2890625]
   [0.21875  ]]

  [[0.859375 ]
   [0.8203125]
   [0.359375 ]
   ...
   [0.375    ]
   [0.296875 ]
   [0.3359375]]

  [[0.8515625]
   [0.6953125]
   [0.28125  ]
   ...
   [0.390625 ]
   [0.3359375]
   [0.2734375]]

  ...

  [[0.3125   ]
   [0.3203125]
   [0.328125 ]
   ...
   [1.2734375]
   [1.1640625]
   [1.03125  ]]

  [[0.3203125]
   [0.328125 ]
   [0.3359375]
   ...
   [1.140625 ]
   [1.0546875]
   [0.921875 ]]

  [[0.3046875]
   [0.3203125]
   [0.3359375]
   ...
   [1.0078125]
   [0.90625  ]
   [0.8203125]]]]


In [92]:
#solution 2 (not work)
from tqdm import tqdm

def imagetensor(imagedir):
  for i, im in tqdm(enumerate(os.listdir(imagedir))):
    image= Image.open(im)
    image= image.convert('HSV')
    if i == 0:
      images= np.expand_dims(np.array(image, dtype= float), axis= 0)
    else:
      image= np.expand_dims(np.array(image, dtype= float), axis= 0)
      images= np.append(images, image, axis= 0)
  return images

image_tensor=imagetensor("C:/Users/Japoka/Driver_safety-master/eyes/OpenFace")

0it [00:00, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'Aaron_Guiel_0001.jpg'